In [2]:
import os, glob
import numpy as np
import pandas as pd
from scipy import stats
from datetime import date, time, datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,  accuracy_score
from keras.models import load_model


In [72]:
def model_selector(flower,week):
    arr =np.array(pd.read_csv("./flower_pre_model/result_best.csv"))
    # flower,week ="Anthurium",1 
    shift2 = (week-1)*7+1
    sample  = arr[(arr[:,0]==flower)*(arr[:,3]==shift2)][0]
    key = "_".join(sample[:4].astype(str))
    fn = glob.glob("./flower_pre_model/"+key+".h5")[0]
    model_info = sample
    return fn, model_info

In [136]:
# df_st = pd.read_csv("./flower_pre_model/result_best.csv")
# flowers = list(df_st["flower"].drop_duplicates())

fn, model_info = model_selector("Eustoma",2)
flower,station,shift1,shift2,test_acc,train_acc = model_info 
fn_weather= glob.glob("../result/merge_weather/" + station + "*.csv")[0]
df_weather = pd.read_csv(fn_weather, encoding="utf-8")
fn_lunar = pd.read_csv("../lunar_done_new.csv", encoding="utf-8") 
model_info 

array(['Eustoma', 'C0X060', 11, 8, 47.14, 83.07], dtype=object)

In [117]:
def prep_rdata(r_data):
    col_fix =  ['T.Max', 'T.Min', 'Precp','Temperature', 'RH', 'StnPres', 'WS', 'WSGust']
    for col in col_fix:
        old = r_data[col]
        new = []
        for i in range(len(old)):
            try:
                new.append(float(old[i]))
                tmp = float(old[i])
            except ValueError:
                new.append(tmp)
        r_data[col] = new
    d_tmp = r_data['T.Max'] - r_data['T.Min']
    r_data["d_tmp"]= d_tmp
    drop_c =["Date",'T.Max', 'T.Min']
    r_data = r_data.drop(columns=drop_c)
    return r_data
def trans_z(r_data):
    recordz = pd.read_csv("./flower_pre_model/weather_trans.csv")
    mean_trans = np.array(recordz[station][0].replace("[","").replace("]","")\
                          .replace(" ","").split(",")).astype(float)
    std_trans = np.array(recordz[station][1].replace("[","").replace("]","")\
                         .replace(" ","").split(",")).astype(float)
    col_x = r_data.columns.to_list()
    for i in range(len(col_x)):
        r_data[col_x[i]]= (r_data[col_x[i]]-[mean_trans[i]])/std_trans[i]
    return r_data
    


In [140]:
# 11, 8
# get df_weather
d_data =  list(df_weather["Date"])

p1 = prep_rdata(df_weather)
p2 = trans_z(p1)
r_data["date"]= d_data
r_data = r_data.join(self.lc_d.set_index("date"), on="date")
r_data = r_data.drop

pred_date = "2016-07-10"
start_index = d_data.index(pred_date)
p3 = p2[(start_index-shift1-shift2):(start_index-shift2)]
date[(start_index-shift1-shift2):(start_index-shift2)]
# p2["date"]=date
# p2

array(['2016-06-21', '2016-06-22', '2016-06-23', '2016-06-24',
       '2016-06-25', '2016-06-26', '2016-06-27', '2016-06-28',
       '2016-06-29', '2016-06-30', '2016-07-01'], dtype=object)

In [144]:
p3 = p3.reset_index(drop=True)

,Temperature,Precp,RH,StnPres,WS,WSGust,d_tmp
0,1.244631,-0.225747,-0.197877,-0.501389,-0.602250,-0.395348,0.426975
1,1.418024,-0.225747,-0.197877,-0.552772,-0.283147,-0.320014,0.045436
2,1.266305,-0.225747,0.082810,-0.672666,-0.176779,-1.337013,-1.137334
3,1.331327,-0.225747,-0.338221,-0.741177,0.142324,0.244985,0.388821
4,1.287979,-0.225747,0.082810,-0.587027,0.035956,-0.320014,-0.069026
5,1.353001,-0.225747,0.082810,-0.501389,-0.495882,-0.395348,-0.526872
6,1.353001,-0.225747,-0.197877,-0.689794,-0.283147,-0.131681,-0.069026
7,1.548068,-0.225747,-0.759253,-0.484261,0.567795,1.977649,0.617744
8,1.461372,-0.225747,-0.759253,-0.141707,0.567795,5.555978,0.159897
9,1.244631,1.417927,-0.197877,-0.158834,-0.070411,0.546318,0.999282


In [112]:
# array(['2016-06-21', '2016-06-22', '2016-06-23', '2016-06-24',
#        '2016-06-25', '2016-06-26', '2016-06-27', '2016-06-28',
#        '2016-06-29', '2016-06-30', '2016-07-01', '2016-07-02',
#        '2016-07-03', '2016-07-04', '2016-07-05', '2016-07-06',
#        '2016-07-07', '2016-07-08', '2016-07-09'], dtype=object)

23.657526881720447